In [ ]:
%load_ext autotime

In [ ]:
# time: 1.88 s

import psycopg2 as pg
import pandas as pd
import pickle
import numpy
import os
import spacy
import string
import time
import lib.functions

# import textacy
from spacy import attrs
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS

from sklearn.base import TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.pipeline import Pipeline

In [ ]:
# time: 1min 51s

texts = {}
texts['wsb'] = pd.read_json('archive/wallstreetbets_submission.json', lines=True)
texts['insub'] = pd.read_json('archive/investing_submission.json', lines=True)
texts['opsub'] = pd.read_json('archive/options_submission.json', lines=True)
texts['secan'] = pd.read_json('archive/SecurityAnalysis_submission.json', lines=True)

all_text = pd.DataFrame()
all_text = all_text.append(texts['insub'])
all_text = all_text.append(texts['wsb'][-150000:])
all_text = all_text.append(texts['opsub'])
all_text = all_text.append(texts['secan'])

all_text['selftext'] = all_text['selftext'].apply(lambda x: x.strip() if type(x) == str else x)
all_text = all_text[all_text['selftext'] != ""]
all_text = all_text[all_text['selftext'] != "[removed]"]
all_text = all_text[all_text['selftext'] != "[deleted]"]
all_text = all_text[all_text['selftext'] != '.']
all_text = all_text[pd.notna(all_text['selftext'])]
all_text.reset_index(drop=True, inplace=True)

all_text.to_csv('archive/sub_reddits.csv', index=False)

In [ ]:
# time: 210 ms

sorttext = sorted(all_text['selftext'])

In [ ]:
all_text.head()

In [ ]:
# Postgres info to connect
connection_args = {
    'host': 'localhost',  # We are connecting to our _local_ version of psql
    'dbname': 'WSB',    # DB that we are connecting to
    'port': 5432          # port we opened on AWS
}

connection = pg.connect(**connection_args)  # What is that "**" there??
cur = connection.cursor()

sqlCreateTable = "create table raw_data (id bigint, type varchar(4), posts text);"

cur.execute(sqlCreateTable)
connection.commit()

# Didn't work.  Created manually.


def make_SQL_string():
    columns = ["post_" + str(i) + " text, " for i in range(58)]

    return "CREATE TABLE posts;"


cur.execute(make_SQL_string())
connection.commit()

In [ ]:
churchill = pd.read_csv('sources/Churchill/Churchill.csv', sep="|", header=None)

In [ ]:
# time: 4.31 s

all_text = pd.read_csv('archive/sub_reddits.csv')
all_text['selftext'] = all_text['selftext'].astype(str)

# Needs additional cleaning

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
docs = [doc for doc in nlp.pipe(all_text['selftext'][:2], batch_size=250, n_threads=4)]

In [ ]:
with open('archive/text_1.pkl', 'wb') as picklefile:
    pickle.dump(docs[0:100000], picklefile)
with open('archive/text_2.pkl', 'wb') as picklefile:
    pickle.dump(docs[100000:200000], picklefile)
with open('archive/text_3.pkl', 'wb') as picklefile:
    pickle.dump(docs[200000:300000], picklefile)
with open('archive/text_4.pkl', 'wb') as picklefile:
    pickle.dump(docs[300000:344748], picklefile)

In [ ]:
with open('archive/text_1.pkl', 'rb') as picklefile:
    all_text = pickle.load(picklefile)
with open('archive/text_2.pkl', 'rb') as picklefile:
    text2 = pickle.load(picklefile)
with open('archive/text_3.pkl', 'rb') as picklefile:
    text3 = pickle.load(picklefile)
with open('archive/text_4.pkl', 'rb') as picklefile:
    text4 = pickle.load(picklefile)

all_text.extend(text2)
all_text.extend(text3)
all_text.extend(text4)

In [ ]:
bow_reprs = [doc.count_by(attrs.LOWER) for doc in all_text]

vocab_keys = set()
for bow_rep in bow_reprs:
    vocab_keys.update(set(bow_rep.keys()))

In [ ]:
pos_reprs = [doc.count_by(attrs.POS) for doc in all_text]
pos_keys = set()
for pos_rep in pos_reprs:
    pos_keys.update(set(pos_rep.keys()))

In [ ]:
def lexeme_filter(lexeme):
    if lexeme.is_digit:
        return(False)
    if lexeme.is_punct:
        return(False)
    if lexeme.is_space:
        return(False)
    if lexeme.like_num:
        return(False)
    if lexeme.like_email:
        return(False)
    return(True)

In [ ]:
vocab_lexemes = [nlp.vocab[vk] for vk in vocab_keys]
vocab_lexemes_filtered = [vl for vl in vocab_lexemes if lexeme_filter(vl)]

lexeme_encoding = {lexeme.lower : i for i,lexeme in enumerate(vocab_lexemes_filtered)}
rev_lexeme_encoding = {i:k for k,i in lexeme_encoding.items()}
lexeme_word_lookup = {lexeme.lower : lexeme.lower_ for lexeme in vocab_lexemes_filtered}
n_words = len(lexeme_encoding)

In [ ]:
def lexeme_lower_bow_to_vec(lexeme_lower_bow, lexeme_encoding):
    bow_vec = numpy.zeros((len(lexeme_encoding,)), dtype=numpy.int64)
    for k,v in lexeme_lower_bow.items():
        try:
            bow_vec[lexeme_encoding[k]] += v
        except KeyError:
            pass
    return(bow_vec)

In [ ]:
tdm = numpy.vstack(lexeme_lower_bow_to_vec(bow_rep, lexeme_encoding)
                   for bow_rep in bow_reprs)

In [ ]:
lexeme_df = pd.DataFrame([key for key in lexeme_encoding])
lexeme_df.columns = ['key']
lexeme_df['col_headers'] = lexeme_df['key'].map(lexeme_encoding)
lexeme_df['key_words'] = lexeme_df['key'].map(lexeme_word_lookup)

In [ ]:
lexeme_df = pd.DataFrame([key for key in lexeme_encoding])

In [ ]:
lexeme_encoding

In [ ]:
tdm_df = pd.DataFrame(tdm)
tdm_df.columns = lexeme_df['key_words']

In [ ]:
tdm_df.head()